In [73]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [74]:
import pandas as pd
import numpy as np
np.random.seed(132)
from functools import lru_cache

import sys

CODE_PATH = '../code'

sys.path.append(CODE_PATH)
import functions

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score
from sklearn.datasets import load_iris, load_boston, load_breast_cancer, load_wine, load_digits
from scipy.optimize import minimize
from sklearn.linear_model import LogisticRegression
from tqdm import tqdm

%pylab inline

Populating the interactive namespace from numpy and matplotlib


# Берем выборку

In [65]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.datasets import fetch_20newsgroups

twenty_train = fetch_20newsgroups(subset='train', shuffle=True, random_state=42)
cv = TfidfVectorizer(max_features=100)
cv = CountVectorizer(max_features=100)

X = cv.fit_transform(twenty_train.data).toarray()
target = twenty_train.target

X_train, X_test, y_train, y_test = train_test_split(X, target, test_size=0.2, random_state=42, stratify=y)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.3, 
                                                  random_state=42, stratify=y_train)
print(X_train.shape, X_test.shape, X_val.shape)

(6335, 100) (2263, 100) (2716, 100)


In [75]:
# dataset = load_breast_cancer()
dataset = load_digits()
df = pd.DataFrame(dataset['data'])
target = dataset['target']

# df = pd.read_csv('../data/model_25_25.csv')
# target = df['target'].astype(int).values
# del df['target']

X_train, X_test, y_train, y_test = train_test_split(df.values, target, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.3, random_state=42)
print(X_train.shape, X_test.shape, X_val.shape)

(1005, 64) (360, 64) (432, 64)


# N случайных дихотомий (без отбора)

In [66]:
attempts_data = []
N_attempts = 5
l = np.unique(target).size
N = 100 # кол-во дихотомий
wtypes = [None, 'accuracy', 'f1', 'confusion_list']
for i in tqdm(range(N_attempts)):
    accs = []
    code_matrix = functions.make_random_dichs(l, N)
    print('Code Matrix shape == ({},{})'.format(l, code_matrix.shape[1]))
    dich_classifiers = functions.train_dichs(code_matrix, X_train, y_train, 
                                             X_test, y_test, LogisticRegression)
    for score_type in wtypes:
        weight_type = None
        preds, num_real_dich = functions.predict_all(X_val, dich_classifiers, code_matrix, score_type, weight_type)
        acc = accuracy_score(preds, y_val)
        accs.append(acc)
    accs.append(num_real_dich)
    attempts_data.append(accs)
#             print(score_type, weight_type, accuracy_score(preds, y_val))

Training dich classifiers:   0%|          | 0/100 [00:00<?, ?it/s]

Code Matrix shape == (20,100)



Training dich classifiers:   0%|          | 0/100 [00:00<?, ?it/s]


Training dich classifiers:   1%|          | 1/100 [00:00<00:12,  7.69it/s]

Code Matrix shape == (20,100)


...


Training dich classifiers:   1%|          | 1/100 [00:00<00:12,  7.77it/s]

Code Matrix shape == (20,100)



Training dich classifiers:   1%|          | 1/100 [00:00<00:13,  7.52it/s]

Code Matrix shape == (20,100)



100%|██████████| 5/5 [02:56<00:00, 35.35s/it]


In [67]:
wtypes += ['num_real_dich']
df_attempts = pd.DataFrame(attempts_data)
df_attempts.columns = ['s{}'.format(i1) for i1 in wtypes]
df_attempts.describe()

,sNone,saccuracy,sf1,sconfusion_list,snum_real_dich
count,5.000000,5.000000,5.000000,5.000000,5.0
mean,0.245214,0.242121,0.136745,0.229971,100.0
std,0.002852,0.007412,0.015048,0.007503,0.0
min,0.241163,0.232695,0.121134,0.220913,100.0
25%,0.244109,0.237113,0.127025,0.224595,100.0
50%,0.245582,0.243004,0.132548,0.229381,100.0
75%,0.246318,0.246318,0.143962,0.236745,100.0
max,0.248895,0.251473,0.159057,0.238218,100.0


# Отбор

In [ ]:
attempts_data = []
N_attempts = 5
l = np.unique(target).size
N = 80 # кол-во дихотомий
wtypes = [None, 'accuracy', 'f1', 'confusion_list']
for i in tqdm(range(N_attempts)):
    accs = []
    code_matrix = functions.make_random_dichs(l, N)
#     print('Code Matrix shape == ({},{})'.format(l, code_matrix.shape[1]))
    dich_classifiers = functions.train_dichs(code_matrix, X_train, y_train, 
                                             X_test, y_test, LogisticRegression)
    for score_type in wtypes:
        preds, num_real_dich = functions.predict_all(X_val, dich_classifiers, 
                                      code_matrix, score_type, weight_type=-1)
        acc = accuracy_score(preds, y_val)
        accs.append(acc)
    accs.append(num_real_dich)
    attempts_data.append(accs)

Training dich classifiers:  26%|██▋       | 21/80 [00:00<00:01, 47.08it/s]

In [ ]:
wtypes += ['num_real_dich']
df_attempts = pd.DataFrame(attempts_data)
df_attempts.columns = ['s{}'.format(i1) for i1 in wtypes]
df_attempts.describe()

# Дихотомии где остались лучшие по критерию

In [339]:
attempts_data = []
N_attempts = 5
l = np.unique(target).size
N = 100 # кол-во дихотомий
num_real_dich = 30 # кол-во дихотомий которые оставляем
wtypes = [None, 'accuracy', 'f1', 'confusion_list']
for i in tqdm(range(N_attempts)):
    accs = []
    code_matrix = functions.make_random_dichs(l, N)
    print('Code Matrix shape == ({},{})'.format(l, code_matrix.shape[1]))
    dich_classifiers = functions.train_dichs(code_matrix, X_train, y_train, 
                                             X_test, y_test, LogisticRegression)
    
    metric_map = np.array([d['f1'] for d in dich_classifiers])
    top_dich = np.sort(np.argsort(metric_map)[-num_real_dich:])
    dich_classifiers = [d for i,d in enumerate(dich_classifiers) if i in top_dich]
    code_matrix = code_matrix.T[top_dich].T
    
    for score_type in wtypes:
        weight_type = None
        preds, _ = functions.predict_all(X_val, dich_classifiers, code_matrix, score_type, weight_type)
        acc = accuracy_score(preds, y_val)
        accs.append(acc)
    accs.append(num_real_dich)
    attempts_data.append(accs)
#             print(score_type, weight_type, accuracy_score(preds, y_val))

Training dich classifiers:   4%|▍         | 4/100 [00:00<00:02, 32.85it/s]

Code Matrix shape == (10,100)



Training dich classifiers:   5%|▌         | 5/100 [00:00<00:02, 45.84it/s]

Code Matrix shape == (10,100)



Training dich classifiers:   6%|▌         | 6/100 [00:00<00:01, 49.51it/s]

...


Training dich classifiers:   6%|▌         | 6/100 [00:00<00:01, 51.44it/s]

Code Matrix shape == (10,100)



Training dich classifiers:   5%|▌         | 5/100 [00:00<00:02, 41.21it/s]

Code Matrix shape == (10,100)



100%|██████████| 5/5 [00:18<00:00,  3.62s/it]00/100 [00:02<00:00, 46.16it/s]


In [340]:
wtypes += ['num_real_dich']
df_attempts = pd.DataFrame(attempts_data)
df_attempts.columns = ['s{}'.format(i1) for i1 in wtypes]
df_attempts.describe()

,sNone,saccuracy,sf1,sconfusion_list,snum_real_dich
count,5.000000,5.000000,5.000000,5.000000,5.0
mean,0.918056,0.920370,0.921759,0.921759,30.0
std,0.011053,0.007781,0.010010,0.008085,0.0
min,0.912037,0.912037,0.909722,0.912037,30.0
25%,0.912037,0.916667,0.918981,0.916667,30.0
50%,0.912037,0.918981,0.921296,0.921296,30.0
75%,0.916667,0.921296,0.921296,0.925926,30.0
max,0.937500,0.932870,0.937500,0.932870,30.0


# Локальный метод оптимизации

In [287]:
def score_function(cur_dich):
    # cluster
    target = np.array([cur_dich[i] for i in y_train])
    return -functions.cluster_score(X_train, target, score_type='trace_w')

In [327]:
def score_function(cur_dich):
    # accuracy
    X = X_train
    y = np.array([cur_dich[i] for i in y_train])
    if y.max() == y.min(): #trivial dich
        return 0
    clf = LogisticRegression()
    clf.fit(X, y)
    y_pred = clf.predict(X_test)
    y_true = np.array([cur_dich[i] for i in y_test])
    accuracy = accuracy_score(y_true, y_pred)
    return accuracy

In [328]:
def score_function(cur_dich):
    # f1
    X = X_train
    y = np.array([cur_dich[i] for i in y_train])
    if y.max() == y.min(): #trivial dich
        return 0
    clf = LogisticRegression()
    clf.fit(X, y)
    y_pred = clf.predict(X_test)
    y_true = np.array([cur_dich[i] for i in y_test])
    accuracy = f1_score(y_true, y_pred)
    return accuracy

In [333]:
code_matrix = functions.make_code_matrix_local(l, 50, score_function, 0)

100%|██████████| 50/50 [00:33<00:00,  1.49it/s]


In [334]:
code_matrix.T.sum()
code_matrix.sum(axis=0)

array([1, 9, 9, 9, 9, 9, 9, 9, 8, 8, 9, 8, 8, 8, 8, 9, 4, 8, 3, 7, 7, 8,
       7, 8, 8, 7, 3, 6, 8, 8, 4, 7, 8, 8, 7, 8, 2, 3, 8, 7, 5, 7, 6, 8,
       7, 8, 7, 8, 8, 7])

In [335]:
attempts_data = []
N_attempts = 5
l = np.unique(target).size
N = 30 # кол-во дихотомий
wtypes = [None, 'accuracy', 'f1', 'confusion_list']
for i in tqdm(range(N_attempts)):
    accs = []
    code_matrix = functions.make_code_matrix_local(l, N, score_function, 0)
    print('Code Matrix shape == ({},{})'.format(l, code_matrix.shape[1]))
    dich_classifiers = functions.train_dichs(code_matrix, X_train, y_train, 
                                             X_test, y_test, LogisticRegression)
    for score_type in wtypes:
        weight_type = None
        preds, num_real_dich = functions.predict_all(X_val, dich_classifiers, code_matrix, score_type, weight_type)
        acc = accuracy_score(preds, y_val)
        accs.append(acc)
    accs.append(num_real_dich)
    attempts_data.append(accs)

100%|██████████| 30/30 [00:20<00:00,  1.45it/s]

Training dich classifiers:  33%|███▎      | 10/30 [00:00<00:00, 89.52it/s]

Code Matrix shape == (10,30)



100%|██████████| 30/30 [00:20<00:00,  1.43it/s]

Training dich classifiers:  23%|██▎       | 7/30 [00:00<00:00, 63.18it/s]

Code Matrix shape == (10,30)



100%|██████████| 30/30 [00:22<00:00,  1.32it/s]

Training dich classifiers:  30%|███       | 9/30 [00:00<00:00, 87.36it/s]

...


100%|██████████| 30/30 [00:18<00:00,  1.66it/s]

Training dich classifiers:  33%|███▎      | 10/30 [00:00<00:00, 96.55it/s]

Code Matrix shape == (10,30)



100%|██████████| 30/30 [00:21<00:00,  1.42it/s]

Training dich classifiers:  30%|███       | 9/30 [00:00<00:00, 87.12it/s]

Code Matrix shape == (10,30)



100%|██████████| 5/5 [01:52<00:00, 22.51s/it]0/30 [00:00<00:00, 71.46it/s]


In [336]:
wtypes += ['num_real_dich']
df_attempts = pd.DataFrame(attempts_data)
df_attempts.columns = ['s{}'.format(i1) for i1 in wtypes]
df_attempts.describe()

,sNone,saccuracy,sf1,sconfusion_list,snum_real_dich
count,5.000000,5.000000,5.000000,5.000000,5.0
mean,0.937037,0.940278,0.940278,0.936111,30.0
std,0.005992,0.005527,0.006423,0.010037,0.0
min,0.928241,0.930556,0.930556,0.918981,30.0
25%,0.935185,0.942130,0.937500,0.937500,30.0
50%,0.937500,0.942130,0.942130,0.937500,30.0
75%,0.939815,0.942130,0.944444,0.942130,30.0
max,0.944444,0.944444,0.946759,0.944444,30.0


In [295]:
l = np.unique(target).size
code_matrix = functions.make_random_dichs(l, 1)

Adding dich: 100%|██████████| 1/1 [00:00<00:00, 5005.14it/s]


In [308]:
functions.make_code_matrix_local(l, 10, score_function, 0)

100%|██████████| 10/10 [00:00<00:00, 13.15it/s]


array([[0, 1, 1, 1, 1, 1, 0, 0, 1, 1],
       [1, 0, 0, 0, 0, 0, 1, 1, 0, 0],
       [1, 1, 0, 0, 1, 1, 1, 1, 0, 0],
       [1, 1, 0, 0, 1, 1, 1, 1, 0, 0],
       [0, 0, 1, 1, 0, 0, 0, 0, 1, 1],
       [1, 1, 0, 0, 1, 1, 1, 1, 0, 0],
       [0, 0, 1, 1, 0, 0, 0, 0, 1, 1],
       [1, 1, 0, 0, 1, 1, 1, 1, 0, 0],
       [1, 1, 0, 0, 1, 1, 1, 1, 0, 0],
       [1, 1, 0, 0, 1, 1, 1, 1, 0, 0]])

In [301]:
l = np.unique(target).size
code_matrix = None

for i in tqdm(range(10)):
    new_dich = np.random.randint(0, 2, l)
    new_dich = functions.make_local_optimal_dichotomy(new_dich.copy(), code_matrix, score_function, verbose=1)
    code_matrix = functions.add_dich(new_dich, code_matrix)

 10%|█         | 1/10 [00:00<00:01,  8.48it/s]

[1 1 0 0 0 1 1 1 0 1]
[1 1 0 0 1 1 1 1 0 1]
[1 1 0 0 1 1 1 1 0 0]
[1 0 0 0 1 1 1 1 0 0]
[1 0 0 0 1 1 1 0 0 0]
[0 0 0 1 1 1 1 0 0 0]
[0 0 0 0 1 1 1 0 0 0]
[1 0 0 0 1 1 1 0 0 0]
[1 1 0 1 0 0 0 0 1 0]
[1 1 0 1 0 0 0 0 1 1]


 30%|███       | 3/10 [00:00<00:00,  9.79it/s]

[1 0 0 1 0 0 0 0 1 1]
[1 0 0 1 0 0 0 0 0 1]
[1 0 0 1 0 1 0 0 0 1]
[1 0 1 1 0 1 0 0 0 1]
[1 0 1 1 0 1 0 0 1 1]
[0 1 1 1 1 0 0 0 0 0]
[0 1 1 1 0 0 0 0 0 0]
[0 1 1 1 0 0 0 1 0 0]
[0 1 1 1 0 0 0 1 1 0]
[0 1 1 1 0 0 0 1 1 1]
[1 0 1 0 1 1 1 0 1 0]
[1 0 0 0 1 1 1 0 1 0]
[1 0 0 0 1 1 1 0 0 0]


 70%|███████   | 7/10 [00:00<00:00, 12.20it/s]

[0 1 1 0 1 0 1 1 0 1]
[0 1 1 0 1 0 1 1 0 0]
[0 1 0 0 1 0 1 1 0 0]
[0 1 1 0 0 0 1 0 1 1]
[0 1 1 1 0 0 1 0 1 1]
[0 1 1 1 0 0 0 0 1 1]
[0 1 1 1 0 0 0 1 1 1]
[0 0 1 0 1 0 1 1 0 1]
[0 0 1 0 1 0 1 1 0 0]
[0 1 1 0 1 0 1 1 0 0]
[0 1 0 0 1 0 1 1 0 0]


100%|██████████| 10/10 [00:00<00:00, 12.18it/s]

[1 1 1 0 1 0 1 1 0 1]
[1 1 1 0 1 0 1 1 0 0]
[1 1 0 0 1 0 1 1 0 0]
[1 1 0 0 1 0 1 0 0 0]
[1 0 0 0 0 1 0 1 1 1]
[1 0 0 1 0 1 0 1 1 1]


In [296]:

for i in tqdm(range(10)):
    cur_dich = np.random.randint(0, 2, l)
    while functions.does_dich_exist(cur_dich, code_matrix):
        cur_dich = np.random.randint(0, 2, l)
    new_dich = functions.make_local_optimal_dichotomy(cur_dich.copy(), code_matrix, score_function, verbose=1)
    code_matrix = np.hstack([code_matrix, new_dich.reshape((-1, 1))])

 20%|██        | 2/10 [00:00<00:00, 13.02it/s]

[1 1 0 0 1 0 1 0 0 1]
[1 1 0 0 1 0 1 0 0 0]
[0 0 1 0 0 0 0 1 1 1]
[0 0 1 1 0 0 0 1 1 1]
[0 1 1 1 0 0 0 1 1 1]
[1 0 0 1 0 1 1 1 0 0]
[1 0 0 1 1 1 1 1 1 0]
[1 1 0 0 1 1 1 0 0 1]


 70%|███████   | 7/10 [00:00<00:00, 17.75it/s]

[1 0 0 0 1 1 1 0 0 1]
[1 0 0 0 1 1 1 0 0 0]
[1 1 1 0 0 0 0 0 0 1]
[1 1 1 1 0 0 0 0 0 1]
[1 0 1 1 0 0 0 0 0 1]
[1 0 1 1 0 1 0 0 0 1]
[1 0 1 1 0 1 0 0 1 1]
[0 0 0 0 1 0 1 0 0 0]
[0 0 0 1 0 0 1 1 0 0]
[0 0 0 1 0 0 0 1 0 0]


 90%|█████████ | 9/10 [00:00<00:00, 15.07it/s]

[0 0 0 1 0 0 0 1 0 1]
[0 0 1 1 0 0 0 1 0 1]
[0 0 1 1 0 1 0 1 0 1]
[0 0 1 1 0 1 0 1 1 1]
[1 1 0 1 1 1 1 1 1 0]
[1 1 0 0 1 1 1 1 1 0]
[0 1 0 0 1 1 1 1 1 0]
[0 1 0 0 1 0 1 1 1 0]
[0 1 0 0 1 0 1 1 0 0]
[0 1 0 1 0 0 0 0 0 0]
[0 1 1 1 0 0 0 0 0 0]


100%|██████████| 10/10 [00:00<00:00, 14.16it/s]

[0 1 1 1 0 0 0 1 0 0]
[0 1 1 1 0 0 0 1 1 0]
[0 1 1 1 0 0 0 1 1 1]


In [297]:
code_matrix

array([[0., 1., 0., 1., 1., 1., 1., 0., 1., 0., 0.],
       [1., 0., 1., 0., 0., 0., 0., 1., 0., 1., 1.],
       [1., 0., 1., 0., 0., 0., 1., 0., 1., 0., 1.],
       [0., 0., 1., 1., 1., 0., 1., 0., 1., 0., 1.],
       [1., 1., 0., 0., 0., 1., 0., 1., 0., 1., 0.],
       [0., 0., 1., 1., 1., 0., 1., 0., 1., 0., 1.],
       [0., 1., 0., 1., 1., 1., 0., 1., 0., 1., 0.],
       [0., 0., 1., 1., 1., 0., 1., 0., 1., 0., 1.],
       [0., 0., 1., 1., 1., 0., 1., 0., 1., 0., 1.],
       [1., 0., 1., 0., 0., 0., 1., 0., 1., 0., 1.]])